In [ ]:
!nvidia-smi

Thu Apr 24 06:00:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
code = r'''
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <iostream>
#include <chrono>
#include <algorithm>

#define SIZE 512

__device__ void swap(int& a, int& b) {
    int tmp = a;
    a = b;
    b = tmp;
}

__global__ void oddEvenSort(int* data, int n) {
    int tid = threadIdx.x;

    for (int i = 0; i < n; i++) {
        int idx = 2 * tid + (i % 2);
        if (idx + 1 < n && data[idx] > data[idx + 1]) {
            swap(data[idx], data[idx + 1]);
        }
        __syncthreads(); // sync between odd/even phases
    }
}

void cpuBubbleSort(int* arr, int n) {
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                std::swap(arr[j], arr[j + 1]);
            }
        }
    }
}

void printArray(const int* arr, int n, const char* label) {
    std::cout << label << ": ";
    for (int i = 0; i < n; ++i) {
        std::cout << arr[i] << " ";
    }
    std::cout << "\n";
}

int main() {
    int h_data[SIZE], h_cpu[SIZE];
    for (int i = 0; i < SIZE; ++i)
        h_data[i] = rand() % 100;

    std::copy(h_data, h_data + SIZE, h_cpu);

    std::cout << "Original Array:\n";
    printArray(h_data, SIZE, "Original");

    // CPU Timing
    auto start_cpu = std::chrono::high_resolution_clock::now();
    cpuBubbleSort(h_cpu, SIZE);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> cpu_duration = end_cpu - start_cpu;
    std::cout << "CPU Time: " << cpu_duration.count() << " ms\n";

    std::cout << "CPU Sorted Array:\n";
    printArray(h_cpu, SIZE, "CPU Sorted");

    // GPU Setup
    int* d_data;
    cudaMalloc((void**)&d_data, SIZE * sizeof(int));
    cudaMemcpy(d_data, h_data, SIZE * sizeof(int), cudaMemcpyHostToDevice);

    // GPU Timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    dim3 grid(1);
    dim3 block(SIZE / 2);  // Each thread compares one pair

    oddEvenSort<<<grid, block>>>(d_data, SIZE);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, start, stop);
    std::cout << "GPU Time: " << gpu_time << " ms\n";

    // Get result
    int h_result[SIZE];
    cudaMemcpy(h_result, d_data, SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    std::cout << "GPU Sorted Array:\n";
    printArray(h_result, SIZE, "GPU Sorted");

    // Validation (optional)
    bool isValid = true;
    for (int i = 0; i < SIZE; ++i) {
        if (h_result[i] != h_cpu[i]) {
            std::cerr << "Mismatch at index " << i << "!\n";
            isValid = false;
            break;
        }
    }

    if (isValid) {
        std::cout << "Validation Passed: CPU and GPU results match.\n";
    } else {
        std::cerr << "Validation Failed: CPU and GPU results do not match.\n";
    }

    std::cout << "Speedup (CPU/GPU): " << cpu_duration.count() / gpu_time << "x\n";

    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return 0;
}


'''
with open('bubblesort.cu', 'w') as f:
    f.write(code)

In [ ]:
!nvcc -arch=sm_75 bubblesort.cu -o bubblesort

In [ ]:
!./bubblesort

Original Array:
Original: 83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 76 4 43 63 13 38 6 40 4 18 28 88 69 17 17 96 24 43 70 83 90 99 72 25 44 90 5 39 54 86 69 82 42 64 97 7 55 4 48 11 22 28 99 43 46 68 40 22 11 10 5 1 61 30 78 5 20 36 44 26 22 65 8 16 82 58 24 37 62 24 0 36 52 99 79 50 68 71 73 31 81 30 33 94 60 63 99 81 99 96 59 73 13 68 90 95 26 66 84 40 90 84 76 42 36 7 45 56 79 18 87 12 48 72 59 9 36 10 42 87 6 1 13 72 21 55 19 99 21 4 39 11 40 67 5 